In [1]:
from feature.base_dataset import *
from feature.after_dataset import *
import random
import pandas as pd
import numpy as np
import os
from glob import glob
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [2]:
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))


In [3]:
train = make_dataset(train_input_list, train_target_list)
test = make_dataset(test_input_list, test_target_list)

train = time_value(train)
test = time_value(test)

train = limit_range(train)
test = limit_range(test)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

test['predicted_weight_g'] = 0

train = col_cumsum(train, "시간당분무량", "일간누적분무량")
train = col_cumsum(train, "시간당백색광량", "일간누적백색광량")
train = col_cumsum(train, "시간당적색광량", "일간누적적색광량")
train = col_cumsum(train, "시간당청색광량", "일간누적청색광량")
train = col_cumsum(train, "시간당총광량", "일간누적총광량")


test = col_cumsum_test(test, "시간당분무량", "일간누적분무량")
test = col_cumsum_test(test, "시간당백색광량", "일간누적백색광량")
test = col_cumsum_test(test, "시간당적색광량", "일간누적적색광량")
test = col_cumsum_test(test, "시간당청색광량", "일간누적청색광량")
test = col_cumsum_test(test, "시간당총광량", "일간누적총광량")


c:\Project_MB\Lettuce-Growth-Environment-Prediction\feature\base_dataset.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum


In [4]:
train_temp, test_temp = diff_temp(train, test)
train_dark, test_dark = none_light(train, test)
train_water, test_water = water(train, test)


In [5]:
train = time_split(train)
test = time_split(test)


In [6]:
pivot_train = pivot_data(train)
pivot_test = pivot_data(test)


In [7]:
train_cumsum = train[['DAT', 'obs_time', '일간누적분무량',
                      '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량', 'Case']]
test_cumsum = test[['DAT', 'obs_time', '일간누적분무량',
                    '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량', 'Case']]


In [8]:
train_cumsum = cumsum_group_max(train_cumsum)
test_cumsum = cumsum_group_max(test_cumsum)

train_rep = group_median(train)
test_rep = group_median(test)

train = concat_df(train_rep, train_cumsum)
test = concat_df(test_rep, test_cumsum)

train['diff_temp'] = train_temp
test['diff_temp'] = test_temp
train['dark'] = train_dark
test['dark'] = test_dark
train['water'] = train_water
test['water'] = test_water
